# Using the USDA MS Acess Databse

# Purpose
The purpose of this notebook will be to break down the microsoft access DB provided by the USDA so we can access the 7,793 food items. 
# Services 
1) Ability to give info and Macro breakdowns of food given name and weight. (Need a good search for this). <br>
2) Abiliy to track users food and display all info pertaining to food tracked.

In [6]:
import requests
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
import datetime
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
import json
from pandas.io.json import json_normalize
from datetime import date
from datetime import datetime

In [7]:
# Ensuring all the files exists
if not os.path.exists( r"..\..\data\raw\USDA\FD_GROUP.xlsx" ):
    print("Missing dataset file")
if not os.path.exists( r"..\..\data\raw\USDA\FOOD_DES.xlsx" ):
    print("Missing dataset file")
if not os.path.exists( r"..\..\data\raw\USDA\NUT_DATA.xlsx" ):
    print("Missing dataset file")
if not os.path.exists( r"..\..\data\raw\USDA\NUTR_DEF.xlsx" ):
    print("Missing dataset file")
if not os.path.exists( r"..\..\data\raw\USDA\WEIGHT.xlsx" ):
    print("Missing dataset file")


In [8]:
# Getting the individual sheets fromm each xlsx
xls1 = pd.ExcelFile(r"..\..\data\raw\USDA\FD_GROUP.xlsx")
FD_GROUP = pd.read_excel(xls1, 'FD_GROUP')
xls2 = pd.ExcelFile(r"..\..\data\raw\USDA\FOOD_DES.xlsx")
FOOD_DES = pd.read_excel(xls2, 'FOOD_DES')
xls3 = pd.ExcelFile(r"..\..\data\raw\USDA\NUT_DATA.xlsx")
NUT_DATA = pd.read_excel(xls3, 'NUT_DATA')
xls4 = pd.ExcelFile(r"..\..\data\raw\USDA\NUTR_DEF.xlsx")
NUTR_DEF = pd.read_excel(xls4, 'NUTR_DEF')
xls5 = pd.ExcelFile(r"..\..\data\raw\USDA\WEIGHT.xlsx")
WEIGHT = pd.read_excel(xls5, 'WEIGHT')

# Important Columns/ Names
NDB_No -> The number/ ID assoicated with each food <br>
FdGrp_Cd -> The number/ ID assoicate with each food group <br>
Nutr_No -> The number/ ID assoicate with each nutrition value <br>

# Challeneges
* Searching Food by name (Will do later)
* A search for a food item will return the food item number. We'll skip the food search by name for now and concentrate and getting and displaying the right infomration when we have a given NDB_No. We'll return a basic food report with key macro nutrients and later on a more in depth food report containing all macros and vitamins valiable. 

# Basic Report 
** What will it return? ** <br></br>
Macronutrients e.g. Protein, Carbs, Fats, Calories, Energy(KJ), Caffeine, Sugar, Fibre, Sodium and Potasium. <br>
1) Per 100g of food item. <br>
2) Split into provide weight options. <br>
3) Given a name and weight, return the food report 

# Basic Report - Per 100g
The code below creates a dataframe of a given food item showcasing a number of basic macro nutrient values per 100g

In [9]:
# Using butter as an example 
NDB_NUM = 1001 

# Getting values for all the macro nutrient values 
foodName = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'Long_Desc'].item()
foodGroupNum = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'FdGrp_Cd'].item()
foodGroupNam = FD_GROUP.loc[FD_GROUP['FdGrp_Cd'] == foodGroupNum, 'FdGrp_desc'].item()
protein = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 203)]['Nutr_Val'].item()
carbs = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 205)]['Nutr_Val'].item()
fat = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 204)]['Nutr_Val'].item()
cals = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 208)]['Nutr_Val'].item()
caffeine = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 262)]['Nutr_Val'].item()
energyKJ = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 268)]['Nutr_Val'].item()
sugar = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 269)]['Nutr_Val'].item()
fibre = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 291)]['Nutr_Val'].item()
sodium = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 307)]['Nutr_Val'].item()
potassium = NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 306)]['Nutr_Val'].item()
weight = 100
mes_DES = '100G Portion'

# Creating the dataFrame 
foodDF = pd.DataFrame(columns=['Item', 'Weight (g)', 'Measure', 'Food Group', 'Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g) ', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)'])
foodDF.loc[0] = np.array([foodName, weight, mes_DES, foodGroupNam, cals, energyKJ, protein, carbs, fat, sugar, fibre, sodium, potassium, caffeine])
foodDF = foodDF.set_index('Item').T

In [10]:
# Printing the example dataFrame
foodDF

Item,"Butter, salted"
Weight (g),100
Measure,100G Portion
Food Group,Dairy and Egg Products
Calories (Kcal),717.0
Energy (KJ),2999.0
Protein (g),0.85
Carbohydrates (g),0.06
Fats (g),81.11
Sugar (g),0.06
Fibre (g),0.0


# Basic Report for all Weight Options
Below the code creates a dictionary of measurement names to dataFrames each of which contain the basic macro nutrient values per given weight of the current food item. 

In [11]:
# Getting the datFrame containg all the weights of a food item 
weightDF = WEIGHT.loc[WEIGHT['NDB_No'] == 1001]

# Creating a list for the dataFrames 
weightDict = {}

# For loop creates each dataFrame a nd appends them to the final list
for x in range (len(WEIGHT.loc[WEIGHT['NDB_No'] == 1001])):
    mes_DES = weightDF['Msre_Desc'].iloc[x]
    G_Weight = weightDF['Gm_Wgt'].iloc[x]
    
    foodName = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'Long_Desc'].item()
    foodGroupNum = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'FdGrp_Cd'].item()
    foodGroupNam = FD_GROUP.loc[FD_GROUP['FdGrp_Cd'] == foodGroupNum, 'FdGrp_desc'].item()
    protein = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 203)]['Nutr_Val'].item()/100)*G_Weight
    carbs = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 205)]['Nutr_Val'].item()/100)*G_Weight
    fat = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 204)]['Nutr_Val'].item()/100)*G_Weight
    cals = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 208)]['Nutr_Val'].item()/100)*G_Weight
    caffeine = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 262)]['Nutr_Val'].item()/100)*G_Weight
    energyKJ = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 268)]['Nutr_Val'].item()/100)*G_Weight
    sugar = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 269)]['Nutr_Val'].item()/100)*G_Weight
    fibre = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 291)]['Nutr_Val'].item()/100)*G_Weight
    sodium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 307)]['Nutr_Val'].item()/100)*G_Weight
    potassium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_NUM) & (NUT_DATA.Nutr_No == 306)]['Nutr_Val'].item()/100)*G_Weight
    
    foodDF = pd.DataFrame(columns=['Item', 'Weight (g)', 'Measure', 'Food Group', 'Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g) ', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)'])
    foodDF.loc[0] = np.array([foodName, G_Weight, mes_DES, foodGroupNam, cals, energyKJ, protein, carbs, fat, sugar, fibre, sodium, potassium, caffeine])
    foodDF = foodDF.set_index('Item').T
    #Creating the dictionary
    weightDict[mes_DES] = foodDF


In [12]:
weightDict

{'cup': Item                       Butter, salted
 Weight (g)                          227.0
 Measure                               cup
 Food Group         Dairy and Egg Products
 Calories (Kcal)                   1627.59
 Energy (KJ)                       6807.73
 Protein (g)                        1.9295
 Carbohydrates (g)                  0.1362
 Fats (g)                         184.1197
 Sugar (g)                          0.1362
 Fibre (g)                             0.0
 Sodium (mg)                       1459.61
 Potassium (mg)                      54.48
 Caffeine (mg)                         0.0,
 'pat (1" sq, 1/3" high)': Item                       Butter, salted
 Weight (g)                            5.0
 Measure            pat (1" sq, 1/3" high)
 Food Group         Dairy and Egg Products
 Calories (Kcal)                     35.85
 Energy (KJ)                        149.95
 Protein (g)                        0.0425
 Carbohydrates (g)                   0.003
 Fats (g)           

# Function returns DF dictionary of reports for all Weights (inc 100g)

In [13]:
def basicFoodRep(NDB_No):
        # Getting the datFrame containg all the weights of a food item 
    weightDF = WEIGHT.loc[WEIGHT['NDB_No'] == NDB_No]

    # Creating a list for the dataFrames 
    weightDict = {}

    # For loop creates each dataFrame a nd appends them to the final list
    for x in range (len(WEIGHT.loc[WEIGHT['NDB_No'] == 1001])+1):

        
        if(x < len(WEIGHT.loc[WEIGHT['NDB_No'] == 1001])):
            mes_DES = weightDF['Msre_Desc'].iloc[x]
            G_Weight = weightDF['Gm_Wgt'].iloc[x]
            divide = 100
        
        if(x > len(WEIGHT.loc[WEIGHT['NDB_No'] == 1001])-1):
            mes_DES = '100G Portion'
            G_Weight = 1
            divide = 1


        foodName = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_No, 'Long_Desc'].item()
        foodGroupNum = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'FdGrp_Cd'].item()
        foodGroupNam = FD_GROUP.loc[FD_GROUP['FdGrp_Cd'] == foodGroupNum, 'FdGrp_desc'].item()
        protein = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 203)]['Nutr_Val'].item()/divide)*G_Weight
        carbs = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 205)]['Nutr_Val'].item()/divide)*G_Weight
        fat = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 204)]['Nutr_Val'].item()/divide)*G_Weight
        cals = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 208)]['Nutr_Val'].item()/divide)*G_Weight
        caffeine = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 262)]['Nutr_Val'].item()/divide)*G_Weight
        energyKJ = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 268)]['Nutr_Val'].item()/divide)*G_Weight
        sugar = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 269)]['Nutr_Val'].item()/divide)*G_Weight
        fibre = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 291)]['Nutr_Val'].item()/divide)*G_Weight
        sodium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 307)]['Nutr_Val'].item()/divide)*G_Weight
        potassium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 306)]['Nutr_Val'].item()/divide)*G_Weight

        foodDF = pd.DataFrame(columns=['Item', 'Weight (g)', 'Measure', 'Food Group', 'Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g) ', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)'])
        foodDF.loc[0] = np.array([foodName, G_Weight, mes_DES, foodGroupNam, cals, energyKJ, protein, carbs, fat, sugar, fibre, sodium, potassium, caffeine])
        foodDF = foodDF.set_index('Item').T
        #Creating the dictionary
        weightDict[mes_DES] = foodDF

    return weightDict
    

# Final Food Report Function
This function returns a simple food report. In order for the function to run the name of the food must be selected and passed as a parameter the weight must also be passed as a parameter. Each food will have different weight measurements that could be selected for example tbsp. If the user wants to enter their own measurement they would select grams or ounces and then enter the amount of the selected measurement that they used. 

In [14]:
def basicWFoodRep(NDB_No, Weight, amount):

        # Food Varaibles 
        foodName = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_No, 'Long_Desc'].item()
        foodGroupNum = FOOD_DES.loc[FOOD_DES['NDB_No'] == NDB_NUM, 'FdGrp_Cd'].item()
        foodGroupNam = FD_GROUP.loc[FD_GROUP['FdGrp_Cd'] == foodGroupNum, 'FdGrp_desc'].item()
        
        # Weight Varaibles
        
        # if the user chooses 100gs 
        if(Weight == '100g'):
            mes_DES = '100g'
            G_Weight = 1
            divide = 1
        # if the user wants to enter a custom number of gs
        elif(Weight== 'g'):
            mes_DES = str(amount) + Weight
            G_Weight = amount 
            divide = 100
            # if the user wants to enter a custom number of oz
        elif(Weight== 'oz'):
            mes_DES = str(amount) + Weight
            G_Weight = amount*28.3495 
            divide = 100
            # if the user selected one of the pre-defined weights from the DB
        else:
            G_Weight = WEIGHT.loc[(WEIGHT['NDB_No'] == NDB_No) & (WEIGHT['Msre_Desc'] == Weight)]
            G_Weight = G_Weight['Gm_Wgt'].iloc[0]
            mes_DES = Weight
            divide = 100
            
        # Macro Varaibles 
        protein = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 203)]['Nutr_Val'].item()/divide)*G_Weight
        carbs = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 205)]['Nutr_Val'].item()/divide)*G_Weight
        fat = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 204)]['Nutr_Val'].item()/divide)*G_Weight
        cals = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 208)]['Nutr_Val'].item()/divide)*G_Weight
        caffeine = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 262)]['Nutr_Val'].item()/divide)*G_Weight
        energyKJ = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 268)]['Nutr_Val'].item()/divide)*G_Weight
        sugar = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 269)]['Nutr_Val'].item()/divide)*G_Weight
        fibre = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 291)]['Nutr_Val'].item()/divide)*G_Weight
        sodium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 307)]['Nutr_Val'].item()/divide)*G_Weight
        potassium = (NUT_DATA[(NUT_DATA.NDB_No == NDB_No) & (NUT_DATA.Nutr_No == 306)]['Nutr_Val'].item()/divide)*G_Weight

                # Weight Varaible 
        if(Weight == '100g'):
            G_Weight = 100
        
        foodDF = pd.DataFrame(columns=['Item', 'Weight (g)', 'Measure', 'Food Group', 'Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g)', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)'])
        foodDF.loc[0] = np.array([foodName, round(G_Weight, 2), mes_DES, foodGroupNam, round(cals, 2), round(energyKJ, 2), round(protein, 2), round(carbs,2), round(fat, 2), round(sugar, 2), round(fibre, 2), round(sodium, 2), round(potassium, 2), round(caffeine, 2)])
        foodDF = foodDF.set_index('Item').T
            
        return foodDF
    

In [15]:
# E.G. 200g of butter example
basicWFoodRep(1001, 'g', 100)

Item,"Butter, salted"
Weight (g),100
Measure,100g
Food Group,Dairy and Egg Products
Calories (Kcal),717.0
Energy (KJ),2999.0
Protein (g),0.85
Carbohydrates (g),0.06
Fats (g),81.11
Sugar (g),0.06
Fibre (g),0.0


In [16]:
# E.G. 7oz of butter example
basicWFoodRep(1001, 'oz', 7.05479)

Item,"Butter, salted"
Weight (g),200.0
Measure,7.05479oz
Food Group,Dairy and Egg Products
Calories (Kcal),1434.0
Energy (KJ),5997.99
Protein (g),1.7
Carbohydrates (g),0.12
Fats (g),162.22
Sugar (g),0.12
Fibre (g),0.0


# Search Option 
Search option: <br>
- User enters a word and the program returns all food items with that word in there name. <br>
- User enters a word e.g. ban so banana would p come up. <br>
- Each entry would prompt a drop down list that the user could choose from. <br> 
- After the user chooses a food. They could then select a size. <br>
- The user would be shown a list of all the sizes e.g. pre defined sizes assigned to each food from the DB, the 100g option or then the user could enter a custom amount in g or ounces. <br>

# Sample Profile (For one Day)
Each Day object will represent a day of eating for a user. It will contain only baisc macros for now as its very easy to add additional ones later. 

In [24]:
class dayM:
    
    def __init__(self, name, ID):
        # Basic varaibles 
        self.name = name
        self.userID = ID
        self.date = date.today()

        # Macro varaibles 
        self.TCals = 0
        self.TProtein = 0
        self.TCarbs = 0
        self.TFat  = 0
        self.TSugar = 0
        self.TFibre = 0
        self.TSodium = 0
        self.TPotassium = 0
        self.TEnergy = 0
        self.TCaffeine  = 0
        
        self.foodNames = []
        self.foodGroups = []
        self.foodItemNum = 0 
        self.foodProfiles = {}
    
    
    def addFood(self, df):
        # resetting index so we can access all of the column data 
        df = df.T.reset_index()
        
        # Adding to the Macro Nutrients 
        self.TCals = self.TCals + float(df['Calories (Kcal)'].item())
        self.TProtein = self.TProtein + float(df['Protein (g)'].item())
        self.TCarbs = self.TCarbs + float(df['Carbohydrates (g)'].item())
        self.TFat  = self.TFat  + float(df['Fats (g)'].item())
        self.TSugar = self.TSugar + float(df['Sugar (g)'].item())
        self.TFibre = self.TFibre + float(df['Fibre (g)'] .item())
        self.TSodium = self.TSodium + float(df['Fibre (g)'].item())
        self.TPotassium = self.TPotassium + float(df['Potassium (mg)'].item())
        self.TEnergy = self.TEnergy + float(df['Energy (KJ)'].item())
        self.TCaffeine  = self.TCaffeine + float(df['Caffeine (mg)'].item())
        
        
        # Adding to the Lists
        self.foodNames.append(df['Item'].item() + ' (' + df['Measure'].item() + ')')
        self.foodGroups.append(df['Food Group'].item())
        self.foodItemNum = self.foodItemNum + 1
        self.foodProfiles[df['Item'].item() + ' (' + df['Measure'].item() + ')'] = df.set_index('Item').T
        
        # Prints out all the macros from all the meals a user has added 
    def displayMacros(self):
        foodDF = pd.DataFrame(columns=['Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g)', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)'])
        foodDF.loc[0] = np.array([self.TCals, self.TEnergy, self.TProtein, self.TCarbs, self.TFat, self.TSugar, self.TFibre, self.TSodium, self.TPotassium, self.TCaffeine])
        foodDF = foodDF[['Calories (Kcal)', 'Energy (KJ)', 'Protein (g)', 'Carbohydrates (g)', 'Fats (g)', 'Sugar (g)', 'Fibre (g)', 'Sodium (mg)', 'Potassium (mg)', 'Caffeine (mg)']].astype(float)
        foodDF['Macronutrient Breakdown'] = 'Totals'
        foodDF = foodDF.set_index('Macronutrient Breakdown').T
        return foodDF.round(2)
    
    # prints out the different food itwms the user added 
    def displayFoodItems(self):
        return list(set(self.foodNames))
        
    # prints out the different food groups the user added
    def displayFoodGroups(self):
        return list(set(self.foodGroups))
        
    # prints out the number of food items a user added 
    def displayItemNum(self):
        return self.foodItemNum
    
    # prints out a dictionary of food items as keys and food reports as values 
    def displayFoodProDict(self):
        return self.foodProfiles

    def displayDate(self):
        return self.date

# User Example 

In [25]:
# Creating user object
dayTemp = dayM('Dan', '100')

# Adding 100g of butter
dayTemp.addFood(basicWFoodRep(1001, '100g', 0))
#Showing macros 
dayTemp.displayMacros()

Macronutrient Breakdown,Totals
Calories (Kcal),717.00
Energy (KJ),2999.00
Protein (g),0.85
Carbohydrates (g),0.06
Fats (g),81.11
Sugar (g),0.06
Fibre (g),0.00
Sodium (mg),0.00
Potassium (mg),24.00
Caffeine (mg),0.00


In [26]:
# Adding another food item 
dayTemp.addFood(basicWFoodRep(1001, 'tbsp', 0))
# Displaying macros 
dayTemp.displayMacros()

Macronutrient Breakdown,Totals
Calories (Kcal),818.81
Energy (KJ),3424.86
Protein (g),0.97
Carbohydrates (g),0.07
Fats (g),92.63
Sugar (g),0.07
Fibre (g),0.00
Sodium (mg),0.00
Potassium (mg),27.41
Caffeine (mg),0.00


In [27]:
# Testing the variuos functions 
# print('Food items added: ', dayTemp.displayFoodItems())
# print('Food groups of items added: ',  dayTemp.displayFoodGroups())
# print('Number of food items added: ', dayTemp.displayItemNum())
# dayTemp.displayFoodProDict().keys()